# Test notebook for Acceptance Test Campaign related to LSST Science Pipelines Release 20.0

This test will be executed on the LSST Science Platform Notebook Aspect, initialized with Science Pipelines release `r20-0-0`.

### Test case LVV-T1946: Verify implementation of measurements in catalogs from coadds
Verify that source measurements in catalogs are in flux units.

In [ ]:
# Confirm that the version of the Science Pipelines is v20_0_0:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
import numpy as np

We will use HSC-RC2, as processed using `v20_0_0_rc1`, which is the pipelines version that was used to create `v20_0_0`.

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-25349":
rc2_repo = '/datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349'

In [ ]:
# Initialize the butler repo:
butler = dafPersist.Butler(rc2_repo)

### Check for both a visit image and a coadd:

In [ ]:
# Select an arbitrary source catalog from a deepCoadd_calexp:
filter = 'HSC-I'
# Coadd
tract = 9615
patch = '5,3'

In [ ]:
dataIdCoadd = {'tract':tract, 'filter':filter, 'patch':patch}
forced_src = butler.get('deepCoadd_forced_src', dataId = dataIdCoadd)

### Check that the schema for the source catalog contains Flux measurements (and that they are in units of "counts")

In [ ]:
fsch = forced_src.schema

#### Forced sources (coadds)

In [ ]:
# Check the units on each entry:
forced_src_flag = []
for a in fsch:
    nnn = a.getField().getName()
    if 'instFlux' in nnn and 'flag' not in nnn:
        flux_units = a.getField().getUnits()
        forced_src_flag.append('count' in flux_units)
        print(f'{nnn:60}..... {flux_units:20}')

In [ ]:
assert(np.all(forced_src_flag)), 'FALSE: not all instFlux entries have units of counts.'
print('All forced_src instFlux entries have units of counts: ',np.all(forced_src_flag))